In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


---

# Processing archive

In [ ]:
# !unzip /content/gdrive/MyDrive/KLTN/dataset/zalo_dataset/Train-Test-Data.zip

Archive:  /content/gdrive/MyDrive/KLTN/dataset/zalo_dataset/Train-Test-Data.zip
 extracting: dataset.zip             
  inflating: public-test.csv         
 extracting: public-test.zip         
  inflating: sample_submission.csv   


In [ ]:
# !unzip /content/dataset.zip

In [ ]:
# !unzip /content/public-test.zip

In [ ]:
# !mv /content/dataset /content/gdrive/MyDrive/KLTN/dataset/zalo_dataset

In [ ]:
# !mv /content/public-test /content/gdrive/MyDrive/KLTN/dataset/zalo_dataset

In [ ]:
# !mv /content/public-test.csv /content/gdrive/MyDrive/KLTN/dataset/zalo_dataset

In [ ]:
# for data in os.listdir("/content/dataset"):
#   for x in os.listdir(os.path.join("/content/dataset", data)):
#     if os.path.isdir(os.path.join("/content/dataset",data,x)):
#       print(os.path.join("/content/dataset",data,x))

/content/dataset/706-M-41/724-M-31
/content/dataset/706-M-41/725-F-28
/content/dataset/706-M-41/726-M-34
/content/dataset/706-M-41/727-F-35
/content/dataset/633-F-28/601-F-27
/content/dataset/633-F-28/602-F-37
/content/dataset/633-F-28/603-M-39
/content/dataset/633-F-28/600-M-29


---

In [ ]:
from tqdm import tqdm
import os
import random
import math
import glob

In [ ]:
const_path = '/content/gdrive/MyDrive'

In [ ]:
zalo_path = const_path + '/KLTN/dataset/zalo_dataset'

In [ ]:
train_path = zalo_path+ "/dataset_fix/train"
test_path = zalo_path+ "/dataset_fix/test"
val_path = zalo_path+ "/dataset_fix/val"

# List generator

In [ ]:
def list_generator(list_path, wav_path, data_type):
    with open(list_path +"/"+ data_type+"_list.txt",'w') as f:
        for speaker_id in os.listdir(wav_path):
          files = glob.glob(os.path.join(wav_path,speaker_id)+ '/**/*.wav', recursive=True)
          for i in range(len(files)):
            files[i] = files[i].replace(wav_path+'/', '')
            f.writelines(str(speaker_id)+" "+files[i].replace("\\","/")+"\n")

In [ ]:
# https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test.txt
def verification_list_generator_veri(list_path, data_type, same_speaker_count, diff_speaker_count):
    with open(list_path + "/veri_val.txt", 'w') as f:
        with open(list_path + "/" + data_type + "_list.txt") as y:
            lines = y.readlines()
        dirs = list(set([x.split()[1] for x in lines]))
        same_speaker_pairs = []
        diff_speaker_pairs = []
        dict = {}
        # Generate same speaker
        while (len(same_speaker_pairs)<same_speaker_count):
          a = random.sample(dirs,2)
          x, y = (hash(a[0]), hash(a[1])) if a[0] < a[1] else (hash(a[1]), hash(a[0]))
          hash_val = round((x + y) * (x + y + 1) / 2 + x)
          if not (dict.get(hash_val)):
              dict[hash_val] = True
              if a[0].split("/")[0] == a[1].split("/")[0]:
                same_speaker_pairs.append((1, a[0],a[1]))

        # Generate different speaker
        while (len(diff_speaker_pairs)<diff_speaker_count):
          a = random.sample(dirs,2)
          x, y = (hash(a[0]), hash(a[1])) if a[0] < a[1] else (hash(a[1]), hash(a[0]))
          hash_val = round((x + y) * (x + y + 1) / 2 + x)
          if not (dict.get(hash_val)):
              dict[hash_val] = True
              if a[0].split("/")[0] != a[1].split("/")[0]:
                diff_speaker_pairs.append((0, a[0],a[1]))


        pairs = same_speaker_pairs + diff_speaker_pairs
        random.shuffle(pairs)

        for i in range(len(pairs)):
            f.writelines(str(pairs[i][0]) + " " + str(pairs[i][1]) + " " + str(pairs[i][2]) + "\n")


In [ ]:
def verification_list_generator(list_path, data_type, same_speaker_count, diff_speaker_count):
    with open(list_path + "/verification_" + data_type + "_list.txt", 'w') as f:
        with open(list_path + "/" + data_type + "_list.txt") as y:
            lines = y.readlines()
        dirs = list(set([x.split()[1] for x in lines]))
        same_speaker_pairs = []
        diff_speaker_pairs = []
        dict = {}
        # Generate same speaker
        while (len(same_speaker_pairs)<same_speaker_count):
          a = random.sample(dirs,2)
          x, y = (hash(a[0]), hash(a[1])) if a[0] < a[1] else (hash(a[1]), hash(a[0]))
          hash_val = round((x + y) * (x + y + 1) / 2 + x)
          if not (dict.get(hash_val)):
              dict[hash_val] = True
              if a[0].split("/")[0] == a[1].split("/")[0]:
                same_speaker_pairs.append((a[0],a[1], 1))

        # Generate different speaker
        while (len(diff_speaker_pairs)<diff_speaker_count):
          a = random.sample(dirs,2)
          x, y = (hash(a[0]), hash(a[1])) if a[0] < a[1] else (hash(a[1]), hash(a[0]))
          hash_val = round((x + y) * (x + y + 1) / 2 + x)
          if not (dict.get(hash_val)):
              dict[hash_val] = True
              if a[0].split("/")[0] != a[1].split("/")[0]:
                diff_speaker_pairs.append((a[0],a[1], 0))


        pairs = same_speaker_pairs + diff_speaker_pairs
        random.shuffle(pairs)

        for i in tqdm(range(len(pairs))):
            f.writelines(str(pairs[i][0]) + " " + str(pairs[i][1]) + " " + str(pairs[i][2]) + "\n")


In [ ]:
list_generator(zalo_path, train_path , "train")
list_generator(zalo_path, test_path , "test")
list_generator(zalo_path, val_path , "val")

In [ ]:
verification_list_generator_veri(zalo_path, "val", 10000, 10000)

In [ ]:
verification_list_generator(zalo_path, "test", 10000, 10000)

100%|██████████| 20000/20000 [00:00<00:00, 214110.35it/s]


## Check duplicate

In [ ]:
with open("/content/gdrive/MyDrive/KLTN/dataset/zalo_dataset/verification_test_list.txt", 'r') as f:
  lines = f.readlines()
data = []
for x in lines:
  tmp = x.split()
  pair = [tmp[0], tmp[1]]
  pair.sort()
  data.append(pair)
for i in tqdm(range(len(data))):
    for j in range(i+1, len(data)):
        if data[i] == data[j]:
            print(f"Same found: {data[i]} vs {data[j]}")
            print(i, j)

100%|██████████| 20000/20000 [00:41<00:00, 482.52it/s] 
